In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline , make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest , chi2

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [6]:
df.drop(['PassengerId','Name','Ticket','Cabin'],axis=1,inplace=True)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('Survived',axis=1),df['Survived'],test_size=0.2,random_state=42)

In [11]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


In [12]:
## Imputation Transformer

trans1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

In [13]:
## One Hot Encoder Transformer

trans2 = ColumnTransformer([
    ('ohe_sex',OneHotEncoder(sparse_output=False,handle_unknown='ignore'),[1,6]),
],remainder='passthrough')

In [16]:
## Scaling Transformer

trans3 = ColumnTransformer([
    ('Scale',MinMaxScaler(),slice(0,10))
])

In [ ]:
## Feature Selection Transformer

trans4 = SelectKBest(score_func=chi2,k=8)

In [19]:
trans5 = DecisionTreeClassifier()

In [20]:
pipe = Pipeline(
    [
        ('trans1',trans1),
        ('trans2',trans2),
        ('trans3',trans3),
        ('trans4',trans4),
        ('trans5',trans5)
    ]
)

In [21]:
## Alternative of pipeline

# pipe = make_pipeline(trans1,trans2,trans3,trans4,trans5)

In [22]:
pipe.fit(x_train,y_train)

,steps,"[('trans1', ...), ('trans2', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('impute_age', ...), ('impute_embarked', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [30]:
pipe.named_steps['trans1'].transformers_[0][1].statistics_

array([29.49884615])

In [ ]:
from sklearn import set_config
set_config(display='diagram')

In [32]:
y_pred = pipe.predict(x_test)

In [33]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)*100

62.56983240223464

In [36]:
## Cross Validation 

from sklearn.model_selection import cross_val_score
cross_val_score(pipe,x_train,y_train,cv=5 , scoring='accuracy').mean()*100

63.91214419383433

In [38]:
## GridSearch for Pipeline

from sklearn.model_selection import GridSearchCV
param_grid = {
    'trans5__max_depth':[1,2,3,4,5,None],
}

grid = GridSearchCV(pipe,param_grid=param_grid,cv=5,scoring='accuracy')

grid.fit(x_train,y_train)

,estimator,Pipeline(step...lassifier())])
,param_grid,"{'trans5__max_depth': [1, 2, ...]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('impute_age', ...), ('impute_embarked', ...)]"


In [40]:
grid.best_score_*100

63.91214419383433

In [42]:
import pickle

pickle.dump(pipe,open('model_pipeline.pkl','wb'))
